In [1]:
!pip install kaggle
!pip install autoviz -q

In [2]:
from google.colab import drive
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
!mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [4]:
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

In [5]:
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
! kaggle datasets download yasserh/wine-quality-dataset

wine-quality-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [7]:
! unzip wine-quality-dataset.zip

Archive:  wine-quality-dataset.zip
replace WineQT.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
# Drop Id column
df = df.drop(columns=['Id'])
df.head()

In [ ]:
# Define features and y
features = df.drop(columns = ['quality'])
y = (df['quality'] > 5)*1
y.head()

# Logistic Regression Classifier with default Regularization

In [ ]:
# Standardize data
X = StandardScaler().fit_transform(features)

# Split data into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=99)

# Define logistic regression classifier without regularization
lr_no_reg = LogisticRegression()

# Fit model with data
lr_no_reg.fit(X_train, y_train)

In [ ]:
# Plot the coefficients obtained from the model
predictors = features.columns
coefficients = lr_no_reg.coef_.ravel()                 # ravel() returns a flattened array
coef = pd.Series(coefficients, predictors).sort_values()

coef.plot(kind='bar', title='Coefficients (no regularization)')
plt.tight_layout()
plt.show()

In [ ]:
y_pred_train = lr_no_reg.predict(X_train)
y_pred_test = lr_no_reg.predict(X_test)

f1_train = f1_score(y_train, y_pred_train)
f1_test = f1_score(y_test, y_pred_test)

print(f'f1 score for training data (default regularization): {f1_train}')
print(f'f1 score for test data (default regularization): {f1_test}')

# logistic Regression with L2 Regularization

In [ ]:
#Write function to collect f1 scores for train and test data while C varies in a logistic regression model
C_array = [0.0001, 0.001, 0.01, 0.1, 1]
train_array = []
test_array = []

for x in C_array:
    lr = LogisticRegression(C = x, random_state=99)
    lr.fit(X_train, y_train)
    y_pred_train = lr.predict(X_train)
    y_pred_test = lr.predict(X_test)
    train_array.append(f1_score(y_train, y_pred_train))
    test_array.append(f1_score(y_test, y_pred_test))

In [ ]:
# Plot how f1 scores change while changing parameter C in logistic regression model
plt.plot(C_array,train_array)
plt.plot(C_array,test_array)
plt.xlabel('log C')
plt.ylabel('f1_score')
plt.legend(labels=['train', 'test'])
plt.xscale('log')
plt.show()

In [ ]:
C_array = np.logspace(-4, -2, 100)
lr_l2_reg = LogisticRegressionCV(Cs = C_array,
                                penalty = 'l2',
                                scoring='f1',
                                cv=5,
                                random_state=99)

lr_l2_reg.fit(X_train, y_train)

print('Best C: ', lr_l2_reg.C_)
print('Best score: ', lr_l2_reg.scores_[1].mean(axis=0).max())

In [ ]:
lr_best_ridge = LogisticRegression(C = lr_l2_reg.C_[0])
lr_best_ridge.fit(X_train, y_train)

y_pred_test = lr_best_ridge.predict(X_test)

print(f'f1 score for test data (L2 regularization): {f1_score(y_test, y_pred_test)}')

In [ ]:
# Plot the coefficients obtained from the model
coefficients = lr_l2_reg.coef_.ravel()
coef = pd.Series(coefficients, predictors).sort_values()

coef.plot(kind='bar', title='Coefficients for tuned L2')
plt.tight_layout()
plt.show()

# Feature Selection using L1 Regularization

In [ ]:
C_array = np.logspace(-4, 2, 100)                         # searching between C=0.01 and 100
lr_l1_reg = LogisticRegressionCV(Cs = C_array,
                                penalty = 'l1',
                                solver='liblinear',
                                scoring='f1',
                                cv=5,
                                random_state=99)
lr_l1_reg.fit(X, y)

print('Best C: ', lr_l1_reg.C_)
print('Best score: ', lr_l1_reg.scores_[1].mean(axis=0).max())

In [ ]:
# Obtain score for test data
lr_best_lasso = LogisticRegression(C = lr_l1_reg.C_[0])
lr_best_lasso.fit(X_train, y_train)

y_pred_test = lr_best_lasso.predict(X_test)

print(f'f1 score for test data (L1 regularization): {f1_score(y_test, y_pred_test)}')

In [ ]:
# Plot the coefficients obtained from the model
coefficients = lr_l1_reg.coef_.ravel()
coef = pd.Series(coefficients, predictors).sort_values()

coef.plot(kind='bar', title='Coefficients for tuned L1')
plt.tight_layout()
plt.show()

# **Conclusions**
The best f1 score for Logistic Regression with defaul regularization (L2, C=1): 0.726 \ The best f1 score for Logistic Regression with L2 regularization (C=0.0034): 0.787 \ The best f1 score for Logistic Regression with L1 regularization (C=0.142): 0.721